# Time Series

In [ ]:
import pandas as pd
import time as t
from copy import deepcopy
from tqdm import tqdm
pd.to_datetime("2020-11-11 24:30:00")

In [ ]:
## Separate Date and Time
df = pd.read_csv("time-log-clean-anon.csv")
print(df[df["Time"]=="2021-04-13 03:00:00"])
print(df.iloc[114960:114980,:])
print(df.iloc[120070:120090,:])

df2 = deepcopy(df)
print(df2, "\n", df2.shape)
print(df2.columns)
print(df2["Time"].iloc[0], "\n====\n", type(df2["Time"].iloc[0]))
df2["Date"] = df2["Time"].apply(lambda x: x.split(" ")[0])
df2["Time"] = df2["Time"].apply(lambda x: x.split(" ")[1])
df2 = df2[["Date", "Time", "Activity"]]
print(df2)

In [ ]:
## Testing
print(df2["Activity"].unique())
vals = ["date", 
 'break', 'breakfast', 'planning', '108', 'lunch', '131a', 'career', 'exercise',
 'dinner', 'CLA', 'fys', 'appt', 'help', 'prep', '165a', '141a', '131b', '154a',
 'UDCE', '302', '141b', '171', '106', '137', '167', 'bike', '168', 'walk', '131c',
 '010', '122a', 'bus', '135', '104', '141c', '130', '330', '141', '162', '160', '174']
tmp = {vals[i]:0 for i in range(len(vals))}  # https://www.geeksforgeeks.org/python-convert-a-list-to-dictionary/
print(tmp)
print(df2["Activity"].value_counts())

In [ ]:
# Some times are messed up and not mult's of 5; find them
# There are also lots of breaks in the middle of the night...
# ...this is because pandas sometimes takes the long way around
for i, row in tqdm(df2.iterrows()):
    #if pd.to_datetime(row.Time).minute%5 != 0:
    if pd.to_datetime(row.Time).hour == 3:
        print(row.Date, row.Time)
        break

In [ ]:
## By time of day:
by_min = pd.DataFrame()

# Group the data by time slot
for i, onetime in tqdm(df2.groupby("Time")):
    #print(i, type(i))
    result = {vals[i]:0 for i in range(len(vals))}
    for act in onetime["Activity"]: 
        result[act] += 1
    result["date"] = i
    by_min = by_min.append(result, ignore_index=True)
print(by_min.iloc[0,:])

In [ ]:
by_min = by_min.groupby(["date"]).sum()
# by_min.index.name = "Time"
by_min

In [ ]:
# Something is off, with breaks in the middle of the night
print(by_min.iloc[:10,:].sum())
print()

In [ ]:
## =================  
# from https://towardsdatascience.com/stacked-bar-charts-with-pythons-matplotlib-f4020e4eb4a7

import matplotlib.pyplot as plt

fields = ['break', 'breakfast', 'planning', '108', 'lunch', '131a', 'career', 'exercise',
 'dinner', 'CLA', 'fys', 'appt', 'help', 'prep', '165a', '141a', '131b', '154a',
 'UDCE', '302', '141b', '171', '106', '137', '167', 'bike', '168', 'walk', '131c',
 '010', '122a', 'bus', '135', '104', '141c', '130', '330', '141', '162', '160', '174']
# TODO:
#colors = ['#1D2F6F', '#8390FA', '#6EAF46', '#FAC748']
labels = ['brk', 'bft', 'pln', '108', 'lun', '131a', 'crr', 'xrc',
 'din', 'cla', 'fys', 'apt', 'help', 'prep', '165a', '141a', '131b', '154a',
 'udce', '302', '141b', '171', '106', '137', '167', 'bike', '168', 'walk', '131c',
 '010', '122a', 'bus', '135', '104', '141c', '130', '330', '141', '162', '160', '174']
# figure and axis
fig, ax = plt.subplots(1, figsize=(12, 50))
# plot bars
left = len(by_min) * [0]
for idx, name in tqdm(enumerate(fields)):
    plt.barh(by_min.index, by_min[name], left = left) #, color=colors[idx])
    left = left + by_min[name]
# title, legend, labels
plt.title('Activity by Time of Day\n', loc='left')
plt.legend(labels, bbox_to_anchor=([0.55, 1, 0, 0]), ncol=4, frameon=False)
plt.xlabel('Time spent')
# remove spines
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
# adjust limits and draw grid lines
# f = plt.figure()
# f.set_size_inches(10,120)
# plt.ylim(-0.5, ax.get_yticks()[-1] + 0.5)
# ax.set_axisbelow(True)
# ax.xaxis.grid(color='gray', linestyle='dashed')
plt.show()

In [ ]:
sta_one_thirty_five_ts = by_min
sta_one_thirty_five_ts = sta_one_thirty_five_ts.drop(['bus', 'CLA', 'dinner', 'exercise', 'fys', 'help', 'lunch', 'planning', 'prep', 'walk', 'UDCE', 'appt', 'bike', 'break', 'breakfast', '174', '160', '131c', '130', '168', '141', '167', '165a', '154a', '302', '162', '131b', '131a', '141b', '141a', '137', '010', '122a', '108', '171', '330', '141c', '106', '104', 'career'], axis = 1)
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10,6
plt.xlabel('date')
plt.ylabel('135')
plt.plot(sta_one_thirty_five_ts)

In [ ]:
#rolling statistics

rolmean = sta_one_thirty_five_ts.rolling(window=12).mean()
rolstd = sta_one_thirty_five_ts.rolling(window=12).std()
print(rolmean, rolstd)

In [ ]:
orig = plt.plot(sta_one_thirty_five_ts, color = 'pink', label = 'Original')
mean = plt.plot(rolmean, color = 'purple', label = 'Rolling Mean')
std = plt.plot(rolstd, color = 'red', label = "Rolling Std")
plt.legend(loc = 'best')
plt.title('Rolling Mean and Std')
plt.show(block=False)

In [ ]:
from statsmodels.tsa.stattools import adfuller

print('Results of the Dickey-Fuller Test:')
dftest = adfuller(sta_one_thirty_five_ts['135'], autolag = 'AIC')

dfoutput = pd.Series(dftest[0:4], index = ['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
    
print(dfoutput)

In [ ]:
sta_one_thirty_five_ts_log = np.log(sta_one_thirty_five_ts)
plt.plot(sta_one_thirty_five_ts_log)

In [ ]:
movingAverage = sta_one_thirty_five_ts_log.rolling(window=12).mean()
movingSTD = sta_one_thirty_five_ts_log.rolling(window=12).std()
plt.plot(sta_one_thirty_five_ts_log)
plt.plot(movingAverage, color = 'pink')

In [ ]:
dfLogMinusMovingAvg = sta_one_thirty_five_ts_log - movingAverage

#Remove NA Values
dfLogMinusMovingAvg.dropna(inplace = True)

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    movingAverage = timeseries.rolling(window=12).mean()
    movingSTD = timeseries.rolling(window=12).std()
    
    orig = plt.plot(timeseries, color = 'pink', label = 'Original')
    mean = plt.plot(movingAverage, color = 'purple', label = 'Rolling Mean')
    std = plt.plot(movingSTD, color = 'red', label = 'Rolling STD')
    plt.legend(loc = 'best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries['135'], autolag = 'AIC')
    dfoutput = pd.Series(dftest[0:4], index = ['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used' ])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    
    
    

In [ ]:
test_stationarity(dfLogMinusMovingAvg)

In [ ]:
exponentialDecayWeightedAverage = sta_one_thirty_five_ts_log.ewm(halflife=12, min_periods=0, adjust = True).mean()
plt.plot(sta_one_thirty_five_ts_log)
plt.plot(exponentialDecayWeightedAverage, color = 'pink')

In [ ]:
dfLogMinusMovingExpDecayAvg = sta_one_thirty_five_ts_log - exponentialDecayWeightedAverage
test_stationarity(dfLogMinusMovingExpDecayAvg)

In [ ]:
dfLogDiffShift = sta_one_thirty_five_ts_log - sta_one_thirty_five_ts_log.shift()
plt.plot(dfLogDiffShift)

In [ ]:
dfLogDiffShift.dropna(inplace = True)
test_stationarity(dfLogDiffShift)

In [ ]:
dfLogDiffShift.isnull().sum()
dfLogDiffShift.dropna(axis=0)

In [ ]:
dfLogDiffShift.dropna(inplace = True)
test_stationarity(dfLogDiffShift)

In [ ]:
from statsmodels.tsa.stattools import acf, pacf

lag_acf = acf(dfLogMinusMovingAvg, nlags = 20)
lag_pacf = pacf(dfLogMinusMovingAvg, nlags=20, method = 'ols')

#Plot ACF
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle='--', color = 'pink')
plt.axhline(y=-1.96/np.sqrt(len(dfLogDiffShift)), linestyle='--', color = 'pink')
plt.axhline(y=1.96/np.sqrt(len(dfLogDiffShift)), linestyle='--', color = 'pink')
plt.title('Autocorrelation Function')

#Plot PACF
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle='--', color = 'pink')
plt.axhline(y=-1.96/np.sqrt(len(dfLogDiffShift)), linestyle='--', color = 'pink')
plt.axhline(y=1.96/np.sqrt(len(dfLogDiffShift)), linestyle='--', color = 'pink')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
#AR model
model = ARIMA(dfLogMinusMovingAvg, order =(2,1,7))
results_AR = model.fit(disp=-1)
plt.plot(dfLogMinusMovingAvg)
plt.plot(results_AR.fittedvalues, color ='red')
plt.title('RSS: %4f'% sum((results_AR.fittedvalues-dfLogMinusMovingAvg['135'])**2))

In [ ]:
import matplotlib.pyplot as plt

# convert into x and y
x = list(range(len(by_min.index)))
y = by_min

# plot the co2 data
fig = plt.figure()
plt.plot(x,y)
plt.ylabel('Activity')
plt.xlabel('Time')

In [ ]:
import numpy as np

# apply fast fourier transform and take absolute values
f=abs(np.fft.fft(y))

# get the list of frequencies
num=np.size(x)
freq = [i / num for i in list(range(num))]

# get the list of spectrums
spectrum=f.real*f.real+f.imag*f.imag
nspectrum=spectrum/spectrum[0]

# plot nspectrum per frequency, with a semilog scale on nspectrum
plt.semilogy(freq,nspectrum)

In [ ]:
test=1/(52*7*24*(60/5))

import pandas as pd
results = pd.DataFrame({'freq': freq, 'nspectrum': nspectrum})
results['period'] = results['freq'] / test
plt.semilogy(results['period'], results['nspectrum'])
results

In [ ]:
# improve the plot by convertint the data into grouped per week to avoid peaks
results['period_round'] = results['period'].round()
grouped_week = results.groupby('period_round')['nspectrum'].sum()
plt.semilogy(grouped_week.index, grouped_week)
plt.xticks([1, 13, 26, 39, 52])

In [ ]:
ft_im = by_min
ft_im = ft_im.drop(['bus', 'CLA', 'dinner', 'exercise', 'fys', 'help', 'lunch', 'planning', 'prep', 'walk'], axis = 1)
ft_im = ft_im.drop(['UDCE', 'appt', 'bike', 'break', 'breakfast'], axis = 1)
ft_im

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = 14,7
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

ss = StandardScaler()
label_quality = LabelEncoder()

ft_im['career'] = label_quality.fit_transform(ft_im['career'])

X = ft_im.drop('career', axis = 1)
y = ft_im['career']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.25, random_state = 42)

X_tr_sc = ss.fit_transform(X_train)
X_te_sc = ss.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_tr_sc, y_train)
importances = pd.DataFrame(data={
    'Attribute': X_train.columns,
    'Importance': model.coef_[0]
})

importances = importances.sort_values(by='Importance', ascending = False)

In [ ]:
plt.bar(x=importances["Attribute"], height = importances['Importance'], color = 'pink')
plt.xticks(rotation = 'vertical')
plt.show()